In [1]:
%load_ext autoreload
%autoreload 2
%run A_simple_net.ipynb

Overwriting init.py
Overwriting run.py
2021-06-08T10:25:10+00:00

CPython 3.6.9
IPython 7.15.0

numpy 1.19.0
matplotlib 3.2.2
pyNN 0.9.4
neo 0.6.1
nixio 1.4.9
pandas 1.1.4

compiler   : GCC 8.4.0
system     : Linux
release    : 4.4.0-87-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 32
interpreter: 64bit
host name  : 96c3276e2566
Git hash   : 4865cf77439cfd2376fcca976cc0983960ee58f2
Git repo   : https://github.com/SpikeAI/2020-11_brainhack_Project7
Git branch : main


In [2]:
%pip install --upgrade sPyNNaker

Requirement already up-to-date: sPyNNaker in /home/spinnaker/sPyNNaker/lib/python3.6/site-packages (1!6.0.0)
You should consider upgrading via the '/home/spinnaker/sPyNNaker/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
%mkdir -p outputs

In [4]:
import datetime
date = datetime.datetime.now().date().isoformat()
date = date.replace('-', '') # HACK
#date = '20201208'
#date = '20201219'

In [5]:
%mkdir -p Results/{date}/

In [6]:
print('date =', date)

date = 20210608


# scanning over single parameters

## initializing parameters

In [7]:
tag = date + '_scan_' + simulator

In [8]:
buildCPUTime, simCPUTime, writeCPUTime = run(verbose=True)

2021-06-08 10:25:20 INFO: Read cfg files: /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/interface/spinnaker.cfg, /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/spynnaker.cfg, /home/spinnaker/.spynnaker.cfg
2021-06-08 10:25:20 INFO: Will search these locations for binaries: /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/common_model_binaries : /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/model_binaries
2021-06-08 10:25:20 WARNING: /home/spinnaker/2020-11_brainhack_Project7/benchmark/reports has 14 old reports that have not been closed
2021-06-08 10:25:20 WARNING: A timestep was entered that has forced sPyNNaker to automatically slow the simulation down from real time by a factor of 10. To remove this automatic behaviour, please enter a timescaleFactor value in your .spynnaker.cfg
2021-06-08 10:25:20 INFO: Setting time scale factor to 10.
2021-06-08 10:25:20 INFO: Setting machine ti

['/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/interface/spinnaker.cfg', '/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/spynnaker.cfg', '/home/spinnaker/.spynnaker.cfg']
0 Creating cell populations...
0 Initialising membrane potential to random values...
0 Connecting populations...
0 Setting up recording...
0 Running simulation...


Adding Splitter selectors where appropriate
|0%                          50%                         100%|
2021-06-08 10:25:20 INFO: Time 0:00:00.008288 taken by SpynnakerSplitterSelector
Adding delay extensions as required
|0%                          50%                         100%|
2021-06-08 10:25:20 INFO: Time 0:00:00.009278 taken by DelaySupportAdder
2021-06-08 10:25:23 INFO: Time 0:00:03.071329 taken by SpallocMaxMachineGenerator
Preallocating resources for Extra Monitor support vertices
|0%                          50%                         100%|
2021-06-08 10:25:32 INFO: Time 0:00:08.612543 taken by PreAllocateResourcesForExtraMonitorSupport
Partitioning graph vertices
|0%                          50%                         100%|
Partitioning graph edges
|0%                          50%                         100%|
2021-06-08 10:25:37 INFO: Time 0:00:04.979479 taken by SpYNNakerSplitterPartitioner
Created spalloc job 6007131
2021-06-08 10:25:37 INFO: Created spalloc job 6

0 Writing data to file...



Getting v for All Cells
|0%                          50%                         100%|


--- Vogels-Abbott Network Simulation ---
Nodes                  : 1
Number of Neurons      : 1000
Excitatory conductance : 4 nS
Inhibitory conductance : 51 nS
Excitatory rate        : 1.03875 Hz
Inhibitory rate        : 0.97 Hz
Build time             : 7.41029 ms
Simulation time        : 67824.3 ms
Writing time           : 224.434 ms


## Running Model and Loading Results

In [9]:
%%writefile scan.py
from run import *

import os
import pandas as pd
verbose = True

def scan(variable, values, tag, verbose=verbose):
    filename = 'outputs/{tag}_{variable}.json'.format(tag=tag, variable=variable)
    try:
        df = pd.read_json(filename)
        print('Loaded file')
    except:
        df = pd.DataFrame([], columns=[variable, 'buildCPUTime', 'simCPUTime', 'writeCPUTime'])
        for i, value in enumerate(values):
            if verbose: print('{variable}---{value}'.format(value=value, variable=variable))
            args = {}
            args[variable] = value
            buildCPUTime, simCPUTime, writeCPUTime = run(**args)
            df.loc[i]= {variable: value, 'buildCPUTime':buildCPUTime, 'simCPUTime':simCPUTime, 'writeCPUTime':writeCPUTime}
        df.to_json(filename)
    return df

Overwriting scan.py


In [10]:
import matplotlib.pyplot as plt

# https://stackoverflow.com/questions/31978948/python-stats-models-quadratic-term-in-regression
def plot(df, do_intercept=True):
    variable = df.columns[0]
    fig, ax = plt.subplots(figsize=(8,8))
    
    for regressor, color in zip(['buildCPUTime', 'simCPUTime', 'writeCPUTime'],
                                ['r', 'g', 'b']):

        import patsy
        if do_intercept:
            y, X = patsy.dmatrices('{regressor} ~ {variable}'.format(regressor=regressor, variable=variable), data=df, return_type='dataframe')
        else:
            y, X = patsy.dmatrices('{regressor} ~ 0 + {variable}'.format(regressor=regressor, variable=variable), data=df, return_type='dataframe')

        import statsmodels.api as sm
        fit = sm.OLS(y, X).fit()
        print(fit.summary())

        if do_intercept: intercept = fit.params.Intercept
        slope = fit.params[variable]

        values = np.array(df[variable])
        if do_intercept:
            ax.plot(values, intercept + slope*values, c=color, lw=2, label=f"{regressor}={slope:.3f}*{variable} + {intercept:.3f}")
        else:
            ax.plot(values, slope*values, c=color, lw=2, label=f"{regressor}={slope:.3f}*{variable}")
        df.plot(x=variable, y=regressor, c=color, lw=1, ls='-.', ax=ax, legend=False)

    ax.set_xlim(0)
    ax.set_ylabel('Time (ms)')
    ax.set_ylim(0)
    ax.legend()
    return fig, ax

## plot as a function of neuron numbers

In [11]:
%%writefile scan_N_pop.py

import sys
tag = sys.argv[1]
print('tag=', tag)

from scan import *

import numpy as np
#N_pops = np.array([int(k)**2 for k in np.linspace(8**2, 45**2, 14, endpoint=True)**.5])
N_pops = np.linspace(8**2, 45**2, 14, endpoint=True).astype(int) # will not work because pyNN requests a "square" population
N_pops = (np.linspace(8**2, 45**2, 14, endpoint=True)**.5).astype(int)**2
N_pops

results_pop = scan('N_pop', N_pops, tag=tag)

Overwriting scan_N_pop.py


In [12]:
%run scan_N_pop.py {tag}

2021-06-08 10:26:28 INFO: Read cfg files: /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/interface/spinnaker.cfg, /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/spynnaker.cfg, /home/spinnaker/.spynnaker.cfg
2021-06-08 10:26:28 INFO: Will search these locations for binaries: /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/common_model_binaries : /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/model_binaries
2021-06-08 10:26:28 WARNING: /home/spinnaker/2020-11_brainhack_Project7/benchmark/reports has 14 old reports that have not been closed
2021-06-08 10:26:28 WARNING: A timestep was entered that has forced sPyNNaker to automatically slow the simulation down from real time by a factor of 10. To remove this automatic behaviour, please enter a timescaleFactor value in your .spynnaker.cfg
2021-06-08 10:26:28 INFO: Setting time scale factor to 10.
2021-06-08 10:26:28 INFO: Setting machine ti

tag= 20210608_scan_spinnaker
N_pop---64
['/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/interface/spinnaker.cfg', '/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/spynnaker.cfg', '/home/spinnaker/.spynnaker.cfg']


2021-06-08 10:26:28 INFO: Starting execution process
2021-06-08 10:26:28 INFO: Simulating for 10000 0.1ms timesteps using a hardware timestep of 1000us
Adding Splitter selectors where appropriate
|0%                          50%                         100%|
2021-06-08 10:26:28 INFO: Time 0:00:00.006873 taken by SpynnakerSplitterSelector
Adding delay extensions as required
|0%                          50%                         100%|
2021-06-08 10:26:28 INFO: Time 0:00:00.006528 taken by DelaySupportAdder
2021-06-08 10:26:31 INFO: Time 0:00:03.125171 taken by SpallocMaxMachineGenerator
Preallocating resources for Extra Monitor support vertices
|0%                          50%                         100%|
2021-06-08 10:26:40 INFO: Time 0:00:08.500880 taken by PreAllocateResourcesForExtraMonitorSupport
Partitioning graph vertices
|0%                          50%                         100%|
Partitioning graph edges
|0%                          50%                         100%|
2021-06

N_pop---196
['/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/interface/spinnaker.cfg', '/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/spynnaker.cfg', '/home/spinnaker/.spynnaker.cfg']


2021-06-08 10:27:38 INFO: Time 0:00:03.341286 taken by SpallocMaxMachineGenerator
Preallocating resources for Extra Monitor support vertices
|0%                          50%                         100%|
2021-06-08 10:27:47 INFO: Time 0:00:08.022476 taken by PreAllocateResourcesForExtraMonitorSupport
Partitioning graph vertices
|0%                          50%                         100%|
Partitioning graph edges
|0%                          50%                         100%|
2021-06-08 10:27:51 INFO: Time 0:00:04.708098 taken by SpYNNakerSplitterPartitioner
Created spalloc job 6007134
2021-06-08 10:27:51 INFO: Created spalloc job 6007134
Waiting for board power commands to complete.
2021-06-08 10:27:51 INFO: Waiting for board power commands to complete.
2021-06-08 10:27:56 INFO: Time 0:00:05.048895 taken by SpallocAllocator
2021-06-08 10:27:56 INFO: Creating transceiver for 10.11.195.17
2021-06-08 10:27:56 INFO: Working out if machine is booted
2021-06-08 10:28:00 INFO: Attempting to 

...

2021-06-08 10:39:07 INFO: Time 0:00:04.445522 taken by SpallocMaxMachineGenerator
Preallocating resources for Extra Monitor support vertices
|0%                          50%                         100%|
2021-06-08 10:39:15 INFO: Time 0:00:08.353892 taken by PreAllocateResourcesForExtraMonitorSupport
Partitioning graph vertices
|0%                          50%                         100%|
Partitioning graph edges
|0%                          50%                         100%|
2021-06-08 10:39:20 INFO: Time 0:00:04.753325 taken by SpYNNakerSplitterPartitioner
Created spalloc job 6007153
2021-06-08 10:39:20 INFO: Created spalloc job 6007153
Waiting for board power commands to complete.
2021-06-08 10:39:20 INFO: Waiting for board power commands to complete.
2021-06-08 10:39:29 INFO: Time 0:00:08.941327 taken by SpallocAllocator
2021-06-08 10:39:29 INFO: Creating transceiver for 10.11.193.1
2021-06-08 10:39:29 INFO: Working out if machine is booted
2021-06-08 10:39:33 INFO: Attempting to b

N_pop---1849
['/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/interface/spinnaker.cfg', '/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/spynnaker.cfg', '/home/spinnaker/.spynnaker.cfg']


2021-06-08 10:40:19 INFO: Time 0:00:02.564616 taken by SpallocMaxMachineGenerator
Preallocating resources for Extra Monitor support vertices
|0%                          50%                         100%|
2021-06-08 10:40:27 INFO: Time 0:00:08.017516 taken by PreAllocateResourcesForExtraMonitorSupport
Partitioning graph vertices
|0%                          50%                         100%|
Partitioning graph edges
|0%                          50%                         100%|
2021-06-08 10:40:32 INFO: Time 0:00:04.749491 taken by SpYNNakerSplitterPartitioner
Created spalloc job 6007156
2021-06-08 10:40:32 INFO: Created spalloc job 6007156
Waiting for board power commands to complete.
2021-06-08 10:40:32 INFO: Waiting for board power commands to complete.
2021-06-08 10:40:37 INFO: Time 0:00:05.046755 taken by SpallocAllocator
2021-06-08 10:40:37 INFO: Creating transceiver for 10.11.196.25
2021-06-08 10:40:37 INFO: Working out if machine is booted
2021-06-08 10:40:41 INFO: Attempting to 

N_pop---2025
['/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/interface/spinnaker.cfg', '/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/spynnaker.cfg', '/home/spinnaker/.spynnaker.cfg']


2021-06-08 10:41:28 INFO: Time 0:00:02.534670 taken by SpallocMaxMachineGenerator
Preallocating resources for Extra Monitor support vertices
|0%                          50%                         100%|
2021-06-08 10:41:36 INFO: Time 0:00:08.281553 taken by PreAllocateResourcesForExtraMonitorSupport
Partitioning graph vertices
|0%                          50%                         100%|
Partitioning graph edges
|0%                          50%                         100%|
2021-06-08 10:41:41 INFO: Time 0:00:04.746619 taken by SpYNNakerSplitterPartitioner
Created spalloc job 6007157
2021-06-08 10:41:41 INFO: Created spalloc job 6007157
Waiting for board power commands to complete.
2021-06-08 10:41:41 INFO: Waiting for board power commands to complete.
2021-06-08 10:41:46 INFO: Time 0:00:05.049399 taken by SpallocAllocator
2021-06-08 10:41:46 INFO: Creating transceiver for 10.11.196.25
2021-06-08 10:41:46 INFO: Working out if machine is booted
2021-06-08 10:41:50 INFO: Attempting to 

In [13]:
fig, ax = plot(results_pop)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/scipy/stats/stats.py:1604: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=14
  "anyway, n=%i" % int(n))
/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/statsmodels/regression/linear_model.py:1728: RuntimeWarning: divide by zero encountered in true_divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/statsmodels/regression/linear_model.py:1729: RuntimeWarning: invalid value encountered in double_scalars
  * (1 - self.rsquared))
/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/statsmodels/regression/linear_model.py:1650: RuntimeWarning: divide by zero encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid


                            OLS Regression Results                            
Dep. Variable:           buildCPUTime   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                       nan
Date:                Tue, 08 Jun 2021   Prob (F-statistic):                nan
Time:                        10:42:35   Log-Likelihood:                 448.42
No. Observations:                  14   AIC:                            -868.8
Df Residuals:                       0   BIC:                            -859.9
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         6.1853        inf          0

KeyError: 'N_pop'

In [ ]:
fig.savefig(tag + '_N_pop.png')

## plot as a function of simtime

In [ ]:
simtimes = np.linspace(500, 3000, 15, endpoint=True)
simtimes

In [ ]:
results_simtime = scan('simtime', simtimes)

In [ ]:
fig, ax = plot(results_simtime, do_intercept=True)      

In [ ]:
fig.savefig(tag + '_simtime.png')

# scanning over two parameters

## Setting-up model and initializing parameters

In [ ]:
tag = date + '_scan-2D_' + simulator

## Running Model and Loading Results

In [ ]:
variables = ['N_pop', 'simtime']
filename = 'outputs/' + tag
for variable in variables: filename +=  '_' + variable
filename += '.json'

In [ ]:
import os
import time
import pandas as pd
verbose = True
def scan(variables, values, tag=tag, verbose=verbose):
    filename = 'outputs/' + tag
    for variable in variables: filename +=  '_' + variable
    filename += '.json'
    try:
        df = pd.read_json(filename)
        print('Loaded file')
    except:
        df = pd.DataFrame([], columns=variables + ['buildCPUTime', 'simCPUTime', 'writeCPUTime'])
        for i, values_ in enumerate(values):
            if verbose: print('{variable}---{values_}'.format(values_=values_, variable=variable))
            args = {}
            for variable, value in zip(variables, values_):
                args[variable] = value
            buildCPUTime, simCPUTime, writeCPUTime = run(**args)
            df.loc[i]= dict(zip(variables + ['buildCPUTime', 'simCPUTime', 'writeCPUTime'], 
                                list(values_) + [buildCPUTime, simCPUTime, writeCPUTime]))

        df.to_json(filename)
        
    return df

## Running time as a function of neuron numbers and simtime

In [ ]:
import numpy as np

In [ ]:
N_trials = 200
N_pops = [int(k)**2 for k in np.linspace(15, 60, N_trials, endpoint=True)]
simtimes = np.linspace(200, 3000, N_trials, endpoint=True)

In [ ]:
np.random.seed(42)
np.random.shuffle(N_pops)
np.random.shuffle(simtimes)
values = zip(N_pops, simtimes)

In [ ]:
results = scan(variables=variables, values=values)
results

## analysis

In [ ]:
results_ext = results.copy()
results_ext['simtime_N_pop'] = results_ext['simtime'] * results_ext['N_pop']
results_ext

In [ ]:
variables_ext = ['N_pop', 'simtime_N_pop']
X = results_ext[variables_ext].values.reshape(-1, len(variables_ext))


https://aegis4048.github.io/mutiple_linear_regression_and_visualization_in_python

See https://patsy.readthedocs.io/en/latest/quickstart.html

In [ ]:
import patsy
import statsmodels.api as sm

for regressor in ['buildCPUTime', 'simCPUTime', 'writeCPUTime']:
    y = results_ext[regressor].values
    #print(variables_ext, X.shape, y.shape)
    y, X = patsy.dmatrices('{regressor} ~ 0 + N_pop + simtime_N_pop'.format(regressor=regressor), data=results_ext, return_type='dataframe')

    fit = sm.OLS(y, X).fit()
    fit.summary()

    print('{regressor} (ms) = {fit0:.3f} * N_pop + {fit1:.3f}/1000 * simtime (ms) * N_pop'.format(regressor=regressor, fit0=fit.params[0], fit1=fit.params[1]*1000))